# **Promedio de valor de Créditos para casas nueva y para vivienda usada**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

## 2022

In [12]:
finance_2022 = df_financiamientos_2022_puebla[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(3104, 4)
   id  mes  modalidad     monto
0   1    1          2  21093.00
1   2    1          2  34318.00
2   3    1          2  43431.00
3   4    1          2  51998.00
4   5    1          2  94648.96


In [13]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7648\219771975.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
27,28,1,Nueva,148572.59,1
28,29,1,Nueva,243566.95,1
29,30,1,Usada,318287.55,1
30,31,1,Nueva,330190.00,1
31,32,1,Usada,385482.13,1
...,...,...,...,...,...
3081,3082,12,Usada,4378500.00,4
3082,3083,12,Usada,9600000.00,4
3084,3085,12,Usada,11916000.00,4
3085,3086,12,Nueva,17350000.00,4


In [14]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023,2T 2023,3T 2023,4T 2023
0,Nueva,"1,484,036.81","1,618,614.46","1,688,152.14","1,862,768.46"
1,Usada,"1,351,099.36","1,748,361.62","1,597,250.91","1,679,717.89"


In [15]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2023,"1,484,036.81"
4,Nueva,2T 2023,"1,618,614.46"
6,Nueva,3T 2023,"1,688,152.14"
8,Nueva,4T 2023,"1,862,768.46"
3,Usada,1T 2023,"1,351,099.36"
5,Usada,2T 2023,"1,748,361.62"
7,Usada,3T 2023,"1,597,250.91"
9,Usada,4T 2023,"1,679,717.89"


In [17]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2022', carpeta='assets\graficas')
fig.show()

## 2023

In [7]:
finance_2022 = df_financiamientos_2023_puebla[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(10837, 4)
   id  mes  modalidad      monto
0   1    1          2   19817.75
1   2    1          2       0.00
2   3    1          2       0.00
3   4    1          2   33908.00
4   5    1          2  277298.00


In [8]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7648\219771975.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
5,6,1,Nueva,1112324.29,1
7,8,1,Nueva,672522.08,1
8,9,1,Nueva,340303.84,1
12,13,1,Usada,1168245.82,1
13,14,1,Usada,481372.87,1
...,...,...,...,...,...
10817,10818,12,Usada,728810.59,4
10819,10820,12,Usada,461268.20,4
10823,10824,12,Usada,619000.00,4
10824,10825,12,Usada,800000.00,4


In [9]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023,2T 2023,3T 2023,4T 2023
0,Nueva,"1,617,135.57","1,721,878.13","1,701,100.83","1,709,858.12"
1,Usada,"1,388,331.92","1,455,266.44","1,511,701.71","1,615,482.02"


In [10]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2023,"1,617,135.57"
4,Nueva,2T 2023,"1,721,878.13"
6,Nueva,3T 2023,"1,701,100.83"
8,Nueva,4T 2023,"1,709,858.12"
3,Usada,1T 2023,"1,388,331.92"
5,Usada,2T 2023,"1,455,266.44"
7,Usada,3T 2023,"1,511,701.71"
9,Usada,4T 2023,"1,615,482.02"


In [11]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2023', carpeta='assets\graficas')
fig.show()

## 2024

In [2]:
finance_2022 = df_financiamientos_2024_puebla[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(2497, 4)
   id  mes  modalidad      monto
0   1    1          2  8574319.0
1   2    1          4  1633800.0
2   3    1          2        0.0
3   4    1          2        0.0
4   5    1          2        0.0


In [3]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7648\219771975.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)


,id,mes,modalidad,monto,trimestre
8,9,1,Nueva,1506407.47,1
9,10,1,Nueva,989730.81,1
10,11,1,Nueva,631457.60,1
11,12,1,Nueva,495053.51,1
25,26,1,Nueva,588699.96,1
...,...,...,...,...,...
2472,2473,3,Usada,1027000.00,1
2473,2474,3,Usada,865429.51,1
2482,2483,3,Usada,996515.48,1
2487,2488,3,Usada,654787.62,1


In [4]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023
0,Nueva,"1,626,835.60"
1,Usada,"1,680,374.92"


In [5]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2023,"1,626,835.60"
3,Usada,1T 2023,"1,680,374.92"


In [6]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2024', carpeta='assets\graficas')
fig.show()